In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [3]:
index_name = "bank-index-50000"
cloud = "aws"
region = "us-east-1"

spec = ServerlessSpec(cloud=cloud, region=region)

In [4]:
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 50000}},
 'total_vector_count': 50000,
 'vector_type': 'dense'}

In [5]:
import openai
import time

embedding_model = "text-embedding-3-small"

# Initialize the OpenAI client
client = openai.OpenAI()

def get_embedding(text, model=embedding_model):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding

In [6]:
import time
start_time = time.time()  # Start timing before the request
# Target vector
query_text = "Customer Henderson CreditScore 599 Age 37Tenure 2Balance 0.0NumOfProducts 1HasCrCard 1IsActiveMember 1EstimatedSalary 107000.88Exited 1Complain 1Satisfaction Score 2Card Type DIAMONDPoint Earned 501"
query_embedding = get_embedding(query_text,model=embedding_model)

In [7]:
# Perform the query using the embedding
query_results = index.query(
    vector=query_embedding,
    include_metadata=True,
    top_k=1
)
# Print the query results along with metadata
print("Query Results:")
for match in query_results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match and 'text' in match['metadata']:
        print(f"Text: {match['metadata']['text']}")
    else:
        print("No metadata available.")

response_time = time.time() - start_time              # Measure response time
print(f"Querying response time: {response_time:.2f} seconds")  # Print response time

Query Results:
ID: 1687, Score: 0.856137872
Text: CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720
Querying response time: 15.37 seconds


In [8]:
relevant_texts = [match['metadata']['text'] for match in query_results['matches'] if 'metadata' in match and 'text' in match['metadata']]

combined_text = "\n".join(relevant_texts)
print(combined_text)

CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720


In [9]:
# Combine texts into a single string, separated by new lines
combined_context = "\n".join(relevant_texts)
#prompt
query_prompt="I have this customer bank record with interesting information on age, credit score and more and similar customers. What could I suggest to keep them in my bank in an email with an url to get new advantages based on the fields for each Customer ID:"
itext=query_prompt+ query_text+combined_context
# Augmented input
print("Prompt for the Generative AI model:", itext)

Prompt for the Generative AI model: I have this customer bank record with interesting information on age, credit score and more and similar customers. What could I suggest to keep them in my bank in an email with an url to get new advantages based on the fields for each Customer ID:Customer Henderson CreditScore 599 Age 37Tenure 2Balance 0.0NumOfProducts 1HasCrCard 1IsActiveMember 1EstimatedSalary 107000.88Exited 1Complain 1Satisfaction Score 2Card Type DIAMONDPoint Earned 501CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720


In [10]:
gpt_model = "gpt-4o"

import time
start_time = time.time()  # Start timing before the request

response = client.chat.completions.create(
  model=gpt_model,
  messages=[
    {
      "role": "system",
      "content": "You are the community manager can write engaging email based on the text you have. Do not use a surname but simply Dear Valued Customer instead."
    },
    {
      "role": "user",
      "content": itext
    }
  ],
  temperature=0,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

response_time = time.time() - start_time              # Measure response time
print(f"Querying response time: {response_time:.2f} seconds")  # Print response time

Subject: Exclusive Benefits Await You at Our Bank!

Dear Valued Customer,

We hope this message finds you well. At [Bank Name], we are constantly striving to enhance your banking experience and provide you with the best possible services tailored to your needs.

We noticed that you have been with us for a while, and we truly value your loyalty. To show our appreciation, we are excited to offer you exclusive benefits that are designed to make your banking experience even more rewarding.

**Here’s what you can look forward to:**

1. **Enhanced Rewards Program:** As a DIAMOND cardholder, you can now earn more points on every transaction. Redeem these points for exciting rewards and experiences.

2. **Personalized Financial Advice:** Our team of financial experts is here to help you make the most of your finances. Schedule a free consultation to explore investment opportunities and savings plans that suit your lifestyle.

3. **Exclusive Offers and Discounts:** Enjoy special discounts and o